<a href="https://colab.research.google.com/github/sohdacafe/Machine_Learning/blob/main/kaggle_House_Prices_Advanced_Regression_Techniques_vol2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [166]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [167]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [169]:
columns = df_train.columns
columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [170]:
# 相関性の低いカラムを削除,objectのカラムは残す。
drop_col = []
columns = df_train.columns
for column in columns:
    if df_train[column].dtypes == object :
        continue
    else:
        corr_v = df_train.corr()[column]['SalePrice']
        print(column, corr_v)
        if corr_v < 0.5 and corr_v > -0.5:
            df_train = df_train.drop(column, axis=1)
            drop_col.append(column)
print(drop_col)

Id -0.021916719443431112
MSSubClass -0.08428413512659523
LotFrontage 0.35179909657067854
LotArea 0.2638433538714063
OverallQual 0.7909816005838047
OverallCond -0.0778558940486776
YearBuilt 0.5228973328794967
YearRemodAdd 0.5071009671113867
MasVnrArea 0.4774930470957107
BsmtFinSF1 0.38641980624215627
BsmtFinSF2 -0.011378121450215216
BsmtUnfSF 0.214479105546969
TotalBsmtSF 0.6135805515591944
1stFlrSF 0.6058521846919166
2ndFlrSF 0.31933380283206614
LowQualFinSF -0.02560613000068015
GrLivArea 0.7086244776126511
BsmtFullBath 0.22712223313149718
BsmtHalfBath -0.016844154297359294
FullBath 0.5606637627484452
HalfBath 0.2841076755947784
BedroomAbvGr 0.1682131543007415
KitchenAbvGr -0.1359073708421417
TotRmsAbvGrd 0.5337231555820238
Fireplaces 0.4669288367515242
GarageYrBlt 0.48636167748786213
GarageCars 0.640409197258349
GarageArea 0.6234314389183598
WoodDeckSF 0.32441344456813076
OpenPorchSF 0.31585622711605577
EnclosedPorch -0.12857795792595636
3SsnPorch 0.04458366533574792
ScreenPorch 0.111

In [171]:
df_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,GrLivArea,FullBath,KitchenQual,TotRmsAbvGrd,Functional,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,856,GasA,Ex,Y,SBrkr,856,1710,2,Gd,8,Typ,NaN,Attchd,RFn,2,548,TA,TA,Y,NaN,NaN,NaN,WD,Normal,208500
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,1262,GasA,Ex,Y,SBrkr,1262,1262,2,TA,6,Typ,TA,Attchd,RFn,2,460,TA,TA,Y,NaN,NaN,NaN,WD,Normal,181500
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,920,GasA,Ex,Y,SBrkr,920,1786,2,Gd,6,Typ,TA,Attchd,RFn,2,608,TA,TA,Y,NaN,NaN,NaN,WD,Normal,223500
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,756,GasA,Gd,Y,SBrkr,961,1717,1,Gd,7,Typ,Gd,Detchd,Unf,3,642,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml,140000
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,1145,GasA,Ex,Y,SBrkr,1145,2198,2,Gd,9,Typ,TA,Attchd,RFn,3,836,TA,TA,Y,NaN,NaN,NaN,WD,Normal,250000


In [172]:
# 各カラムのカテゴリ変数数をカウント
columns = df_train.columns
for column in columns:
    print(column, df_train[column].nunique())

MSZoning 5
Street 2
Alley 2
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
LandSlope 3
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
OverallQual 10
YearBuilt 112
YearRemodAdd 61
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 4
ExterQual 4
ExterCond 5
Foundation 6
BsmtQual 4
BsmtCond 4
BsmtExposure 4
BsmtFinType1 6
BsmtFinType2 6
TotalBsmtSF 721
Heating 6
HeatingQC 5
CentralAir 2
Electrical 5
1stFlrSF 753
GrLivArea 861
FullBath 4
KitchenQual 4
TotRmsAbvGrd 12
Functional 7
FireplaceQu 5
GarageType 6
GarageFinish 3
GarageCars 5
GarageArea 441
GarageQual 5
GarageCond 5
PavedDrive 3
PoolQC 3
Fence 4
MiscFeature 4
SaleType 9
SaleCondition 6
SalePrice 663


In [174]:
df_train.groupby('LandSlope')['SalePrice'].mean()

LandSlope
Gtl    179956.799566
Mod    196734.138462
Sev    204379.230769
Name: SalePrice, dtype: float64

In [175]:
# 各カテゴリ変数別の平均値の最大/最小値の差を確認、
# 差が小さいものは相関の無いカテゴリ変数としてはじく。
columns = df_train.columns
for column in columns:
    max_v = df_train.groupby(column)['SalePrice'].mean().max()
    min_v = df_train.groupby(column)['SalePrice'].mean().min()
    ratio_v = max_v/min_v
    if ratio_v < 1.5:
        df_train = df_train.drop(column, axis=1)
        print(column, ratio_v)

Street 1.391273084552582
Alley 1.374585583248161
LotShape 1.4556986448970108
Utilities 1.3160069586890148
LotConfig 1.2651581734290243
LandSlope 1.1357127447382056
BldgType 1.4463952450616677
BsmtFinType2 1.389415723675938
Fence 1.332432071149379


In [176]:
# 欠損値の補完
df_train = df_train.fillna(df_train.mean())
# 欠損値の多いカラムは削除し、欠損値を削除
columns = df_train.columns
for column in columns:
    lack_cont = df_train[column].isnull().sum()
    if lack_cont > 50:
        df_train = df_train.drop(column, axis=1)
df_train = df_train.dropna()

In [182]:
# 'SalePrice'の平均値に応じて、カテゴリごとのポイント付け
columns = df_train.columns
for column in columns:
    df_train.groupby(column)['SalePrice'].mean()
    uniques = df_train[column].unique()
    for unique in uniques:
        unique_v = df_train.groupby(column)['SalePrice'].mean()[unique]
        df_train[column].replace(unique, unique_v, inplace=True)
        # 値をinplace=Trueで置き換え

In [183]:
df_train

,MSZoning,LandContour,Neighborhood,Condition1,Condition2,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,GrLivArea,FullBath,KitchenQual,TotRmsAbvGrd,Functional,GarageCars,GarageArea,PavedDrive,SaleType,SaleCondition,SalePrice
0,193131.356115,181720.727918,197738.959459,186163.942670,182853.190272,210864.718535,207980.156550,229069.441860,196840.416667,172640.115036,181382.885447,214969.808765,215388.943089,205536.179954,232190.561587,185632.233871,225825.271293,202746.278689,183529.563748,165367.475132,235108.075061,179500.000000,183336.795963,215599.644628,187047.506006,187900.553420,208500.0,229580.400000,215736.723951,212885.609756,216550.144444,184751.371882,185528.612863,171750.000000,187661.02454,175273.874286,177005.540517,208500
1,193131.356115,181720.727918,238772.727273,143595.053333,182853.190272,178453.185879,161483.587601,163831.969697,159868.500000,172640.115036,181382.885447,150636.879070,151061.861244,158320.164862,145797.213548,185632.233871,150479.505600,202746.278689,183529.563748,256131.082707,161573.068182,183250.000000,183336.795963,215599.644628,187047.506006,187900.553420,183250.0,183250.000000,215736.723951,141189.170213,162695.483461,184751.371882,185528.612863,175657.142857,187661.02454,175273.874286,177005.540517,181500
2,193131.356115,181720.727918,197738.959459,186163.942670,182853.190272,210864.718535,207980.156550,242630.000000,215068.888889,172640.115036,181382.885447,214969.808765,215388.943089,205536.179954,232190.561587,185632.233871,225825.271293,202746.278689,183529.563748,192789.657895,235108.075061,171625.000000,183336.795963,215599.644628,187047.506006,187900.553420,211750.0,219066.666667,215736.723951,212885.609756,162695.483461,184751.371882,185528.612863,230000.000000,187661.02454,175273.874286,177005.540517,223500
3,193131.356115,181720.727918,210824.740000,186163.942670,182853.190272,210864.718535,207980.156550,134387.500000,139367.307692,172640.115036,181382.885447,150548.532338,166041.666667,158320.164862,145797.213548,185632.233871,132470.668966,140667.435185,209671.781250,165367.475132,161573.068182,164522.222222,183336.795963,158137.555556,187047.506006,187900.553420,129500.0,194333.333333,135628.763407,212885.609756,197331.041009,184751.371882,308772.988889,214404.333333,187661.02454,175273.874286,147142.890000,140000
4,193131.356115,181720.727918,335295.317073,186163.942670,182853.190272,210864.718535,274961.497006,210766.666667,187872.314815,172640.115036,181382.885447,214969.808765,215388.943089,205536.179954,232190.561587,185632.233871,225825.271293,202746.278689,183529.563748,206643.420814,235108.075061,170066.666667,183336.795963,215599.644628,187047.506006,187900.553420,205100.0,250000.000000,215736.723951,212885.609756,257784.569444,184751.371882,308772.988889,308250.000000,187661.02454,175273.874286,177005.540517,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,193131.356115,181720.727918,192857.220779,186163.942670,182853.190272,210864.718535,161483.587601,210614.720000,187872.314815,172640.115036,181382.885447,214969.808765,215388.943089,158320.164862,145797.213548,185632.233871,225825.271293,202746.278689,183529.563748,165367.475132,170517.366197,173700.000000,183336.795963,215599.644628,187047.506006,187900.553420,173700.0,173500.000000,215736.723951,141189.170213,197331.041009,184751.371882,185528.612863,175657.142857,187661.02454,175273.874286,177005.540517,175000
1456,193131.356115,181720.727918,189050.068493,186163.942670,182853.190272,178453.185879,161483.587601,164226.666667,190533.333333,172640.115036,181382.885447,177679.281553,170034.933333,265583.625000,145797.213548,185632.233871,150479.505600,202746.278689,183529.563748,165367.475132,161573.068182,21

In [ ]:
# 線形回帰

In [184]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [186]:
t = df_train['SalePrice'].values
X = df_train.drop(labels=['SalePrice'], axis=1)
X_train, X_test,t_train, t_test = train_test_split(X, t, test_size=0.3, random_state=0) 

In [187]:
LR_model = LinearRegression()
LR_model.fit(X_train, t_train)

LinearRegression()

In [188]:
print(f'train score:{LR_model.score(X_train, t_train)}')
print(f'test score:{LR_model.score(X_test, t_test)}')

train score:0.9597916684860137
test score:0.9492394329035468


In [ ]:
# submit

In [189]:
df_train.columns

Index(['MSZoning', 'LandContour', 'Neighborhood', 'Condition1', 'Condition2',
       'HouseStyle', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', 'GrLivArea', 'FullBath', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'GarageCars', 'GarageArea', 'PavedDrive',
       'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [190]:
df_test = df_test[['MSZoning', 'LandContour', 'Neighborhood', 'Condition1', 'Condition2',
       'HouseStyle', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', 'GrLivArea', 'FullBath', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'GarageCars', 'GarageArea', 'PavedDrive',
       'SaleType', 'SaleCondition']]

In [191]:
df_test = df_test.fillna(df_train.mean())

In [192]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1459 non-null   object 
 1   LandContour    1459 non-null   object 
 2   Neighborhood   1459 non-null   object 
 3   Condition1     1459 non-null   object 
 4   Condition2     1459 non-null   object 
 5   HouseStyle     1459 non-null   object 
 6   OverallQual    1459 non-null   int64  
 7   YearBuilt      1459 non-null   int64  
 8   YearRemodAdd   1459 non-null   int64  
 9   RoofStyle      1459 non-null   object 
 10  RoofMatl       1459 non-null   object 
 11  Exterior1st    1459 non-null   object 
 12  Exterior2nd    1459 non-null   object 
 13  MasVnrType     1459 non-null   object 
 14  ExterQual      1459 non-null   object 
 15  ExterCond      1459 non-null   object 
 16  Foundation     1459 non-null   object 
 17  BsmtQual       1459 non-null   object 
 18  BsmtCond

In [194]:
# 'SalePrice'の平均値に応じて、カテゴリごとのポイント付け
columns = df_test.columns
for column in columns:
    uniques = df_test[column].unique()
    for unique in uniques:
        df_test[column].replace(unique, unique_v, inplace=True)
        # 値をinplace=Trueで置き換え
pred = LR_model.predict(df_test)

In [195]:
# 予測値入力、提出ファイル出力
df_gen = pd.read_csv('sample_submission.csv')
df_gen['SalePrice'] =pred
df_gen.to_csv('sample_submission_3.csv', index=False)